In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [5]:
fashion_mnist = keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
train_images = train_images/255.0
test_images = test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [12]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [13]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter',min_value = 32,max_value = 128,step = 16),
          kernel_size = hp.Choice('conv_1_kernel',values = [3,5]),
          activation = 'relu',
          input_shape = (28,28,1)
      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_2_filter',min_value = 32,max_value = 64,step = 16),
          kernel_size = hp.Choice('conv_2_kernel',values = [3,5]),
          activation = 'relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units',min_value = 32,max_value = 128,step = 16),
          activation = 'relu'
      ),
      keras.layers.Dense(10,activation = 'softmax')

  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',values = [1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [15]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [16]:
tuner_search = RandomSearch(build_model,
                            objective = 'val_accuracy',
                            max_trials = 5,
                            directory = 'output',
                            project_name = 'Mnist Fashion'
                            )

In [17]:
tuner_search.search(train_images,train_labels,epochs = 3,validation_split = 0.1)

Trial 5 Complete [00h 07m 23s]
val_accuracy: 0.9110000133514404

Best val_accuracy So Far: 0.9110000133514404
Total elapsed time: 00h 55m 48s


In [18]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 112)       2912      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        32288     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 80)                1239120   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 1275130 (4.86 MB)
Trainable params: 1275130 (4.86 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.fit(train_images, train_labels, epochs = 15, validation_split = 0.1, initial_epoch = 3)

Epoch 4/15
1688/1688 [==============================] - 132s 77ms/step - loss: 0.1666 - accuracy: 0.9367 - val_loss: 0.2567 - val_accuracy: 0.9065
Epoch 5/15
1688/1688 [==============================] - 130s 77ms/step - loss: 0.1293 - accuracy: 0.9511 - val_loss: 0.2852 - val_accuracy: 0.9015
Epoch 6/15
1688/1688 [==============================] - 126s 74ms/step - loss: 0.1004 - accuracy: 0.9631 - val_loss: 0.3061 - val_accuracy: 0.9112
Epoch 7/15
1688/1688 [==============================] - 124s 73ms/step - loss: 0.0754 - accuracy: 0.9722 - val_loss: 0.3329 - val_accuracy: 0.9100
Epoch 8/15
1688/1688 [==============================] - 125s 74ms/step - loss: 0.0598 - accuracy: 0.9780 - val_loss: 0.3773 - val_accuracy: 0.9127
Epoch 9/15
1688/1688 [==============================] - 124s 74ms/step - loss: 0.0436 - accuracy: 0.9840 - val_loss: 0.3856 - val_accuracy: 0.9055
Epoch 10/15
1688/1688 [==============================] - 124s 74ms/step - loss: 0.0379 - accuracy: 0.9861 - val_loss: 